Импорт библиотек

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import os
import pytorch_lightning
import pytorchvideo.data
import torch.utils.data
import pandas as pd
import csv
import json
from typing import List
import torch
import torch.nn.functional as F
import torchvision.transforms as T
from PIL import Image
from pytorchvideo.data.encoded_video import EncodedVideo
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from ipywidgets import Video
import torch
import torch.nn as nn
import torch.nn.functional as F

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Вспомогательные функции

In [62]:
import datetime

def get_duration(filename):
    video = cv2.VideoCapture(filename)

    duration = video.get(cv2.CAP_PROP_POS_MSEC)
    frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
    
    video = EncodedVideo.from_path(filename)

    return int(video.duration), int(frame_count)

def cast_pred_to_timestamps(pred, file):
    timestamps = []
    is_open = False
    for i in range(0, pred.size):
        if (pred[i] == 0):
            if (i != 0 and pred[i-1] == 0 and i != 1 and pred[i - 2] == 0):
                is_open = False
        elif (is_open):
            continue
        else:
            is_open = True
            timestamps.append(str(datetime.timedelta(seconds=i))[2:])
            
    s = file
    s += ';['
    for i in range(len(timestamps)):
        s += timestamps[i]
        if (i != len(timestamps) - 1):
            s += ','
    s += ']'
    return s

Директории. В DATA_DIR вставить вашу директорию.

In [63]:
train_dir = 'data/train'
val_dir = 'data/val'

DATA_DIR = val_dir
OUTPUT_DIR = 'result'

files = os.listdir(DATA_DIR)[:-1]

Решение с нулями

In [64]:
#sol zeros
accur = 0

res = 'video_name;violation_timestamp\n'
for file in files:
    dur, frame_count = get_duration(os.path.join(DATA_DIR, file))
    
    pred = np.zeros(dur)
    
    res += cast_pred_to_timestamps(pred, file) + '\n'

res = res[:-1]

with open(os.path.join(OUTPUT_DIR, 'zeros.txt'), 'w') as f:
    f.write(res)

Решение 1. YOLO.

In [65]:
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator

model = YOLO("yolov8n.yaml").load("yolov8n.pt")

def predict(filename):
    dur, frame_count = get_duration(filename)
    results = model(filename, verbose=False, stream=True, conf=0.6)
    fps = frame_count/dur
    predict = np.zeros(dur)
    i = 0
    print(frame_count)
    for result in tqdm(results):
        boxes = result.boxes
        if 0. in boxes.cls:
            predict[int(i // fps)] = 1.
        i += 1
    return predict

res = 'video_name;violation_timestamp\n'

for file in files:
    pred = predict(os.path.join(DATA_DIR, file))
    
    res += cast_pred_to_timestamps(pred, file) + '\n'

res = res[:-1]

with open(os.path.join(OUTPUT_DIR, 'first_sol.txt'), 'w') as f:
    f.write(res)

Transferred 355/355 items from pretrained weights
8992


8992it [01:35, 94.64it/s]


4305


4305it [00:45, 93.96it/s]


424


...

236it [00:02, 96.82it/s]


139


139it [00:01, 94.69it/s]


204


204it [00:02, 97.11it/s]


Решение 3. YOLO + случайный лес.

In [ ]:
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator
from sklearn.ensemble import RandomForestClassifier

model = YOLO("yolov8n.yaml").load("yolov8n.pt")

df_train = pd.read_csv("objects.csv")

X_train = df_train.values[:, 2:].astype(int)
y_train = df_train.values[:, 1].astype(int)

classifier = RandomForestClassifier(class_weight={1: 9.2, 0: 1})

classifier.fit(X_train, y_train)

def predict(filename):
    dur, frame_count = get_duration(filename)
    results = model(filename, verbose=False, stream=True, conf=0.6)
    fps = frame_count/dur
    predict = np.zeros(dur)
    i = 0
    print(frame_count)
    for result in tqdm(results):
        boxes = result.boxes  # Boxes object for bounding box outputs    
        if (len(boxes) > 0):
            arr = np.zeros(80).astype(int)
            for cl in boxes.cls:
                arr[int(cl.item())] = 1
            predict[int(i // fps)] = classifier.predict([arr]).item()
        i += 1
    return predict

res = 'video_name;violation_timestamp\n'
for file in files:
    pred = predict(os.path.join(DATA_DIR, file))
    
    res += cast_pred_to_timestamps(pred, file) + '\n'

res = res[:-1]

with open(os.path.join(OUTPUT_DIR, 'second_sol.txt'), 'w') as f:
    f.write(res)

Transferred 355/355 items from pretrained weights
8992


1075it [00:13, 72.04it/s]